In [1]:
# machines = [("seung-titan01", :auto), ("seung-titan02", :auto), ("seung-titan03", :auto)]
# addprocs(machines; topology=:master_slave)
# addprocs(6)
# @show nworkers()

In [ ]:
ENV["PYTHON"]="/usr/people/jingpeng/lib/anaconda3/bin/python"
Pkg.build("PyCall")
using PyPlot

INFO: Building Conda
INFO: Building PyCall


In [ ]:
@everywhere using RealNeuralNetworks
RealNeuralNetworks.Neurons
@everywhere using RealNeuralNetworks.Neurons
@everywhere using RealNeuralNetworks.Neurons.Segments
using Plots
using PyPlot
@everywhere using OffsetArrays
using JLD2
@everywhere using JLD2
include(joinpath(Pkg.dir(),"RealNeuralNetworks/src/Utils/PlotRecipes.jl"))

@everywhere const VOXEL_SIZE = (1000,1000,1000)
@everywhere const GAUSSIAN_FILTER_STD = 4.0

function coloring(dm)
    img = Array{RGB{Float64}}(size(dm))
    for i in eachindex(dm)
        img[i] = get(ColorSchemes.jet, dm[i])
    end 
    img
end 

using Colors, ColorSchemes

function plot(neuron::Neuron)
    segmentList = neuron.segmentList
    plotly()
    for branch in segmentList
        nodeList = Neurons.Segments.get_node_list(branch)
        #@show length(nodeList)
        #@show nodeList
        x = map(n->n[1], nodeList[1:10:end])
        y = map(n->n[2], nodeList[1:10:end])
        z = map(n->n[3], nodeList[1:10:end])
        plot!(x,y,z) #color=rand(Colors.RGB))
    end 

    root = Neurons.get_root_node(neuron)
    plot!([root[1]], [root[2]], [root[3]], m=(2, :circle), leg=false)
end 

function plot_arbor_density_map(densityMap::Array)
    fig = PyPlot.figure()
    fig[:add_subplot](2,2,1)
    xy = maximum(densityMap, 3)[:,:,1]
    PyPlot.imshow(xy, "jet")
    # colorbar()
    fig[:add_subplot](2,2,2)
    xz = maximum(densityMap, 2)[:,1,:]
    PyPlot.imshow(xz, "jet")
    fig[:add_subplot](2,2,3)
    yz = maximum(densityMap, 1)[1,:,:] |> rotl90
    PyPlot.imshow(yz, "jet")
    colorbar()
end 

function plot_mask(mask::Array)
    fig = PyPlot.figure()
    fig[:add_subplot](2,2,1)
    xy = maximum(mask, 3)[:,:,1]
    PyPlot.imshow(xy, "gray")
    # colorbar()
    fig[:add_subplot](2,2,2)
    xz = maximum(mask, 2)[:,1,:]
    PyPlot.imshow(xz, "gray")
    fig[:add_subplot](2,2,3)
    yz = maximum(mask, 1)[1,:,:] |> rotl90
    PyPlot.imshow(yz, "gray")
end 

In [ ]:
cellIdList = Vector{Int}()
# since the valid cellIdList is latest and the consensus is last month, we should exclude the cells completed last month
invalidIdSet = IntSet([78045, 77845, 77912, 78059, 78065, 76871, 77819, 78060, 76876, 78058, 76692, 77937, 78062, 78077, 77933, 76896, 78137, 77938, 78090, 78049, 76899, 78050, 77940, 78139, 76902, 78130, 78159, 78102, 77641, 77927, 78095, 77942, 78047, 78072, 78093, 78051, 78166, 77907, 76923, 77709, 78042, 76932, 76933, 78094, 78113, 78144, 76936, 78188, 78054, 78053, 76938, 78079, 78075, 78157, 77999, 78074, 77945, 78087, 78061, 78120, 78111, 78146, 78100, 78096, 78141, 77893, 77052, 78191, 78117, 78163, 78153, 77050, 78131, 78160, 78190, 78206, 78170, 78169, 78109, 78154, 78108, 78147, 78187, 78078, 78196, 76776, 78089, 76940, 76943, 76950, 78103, 78076, 78178, 78165, 76966, 78162, 76967, 76972, 78124, 78164, 78174, 78184, 77024, 77802, 77803, 77025, 77618, 78129, 78156, 78186, 78086, 76697, 78118, 77806, 78179, 78181, 77811, 77862, 78082, 77782, 78080, 77864, 77869, 78134, 78110, 78132, 77871, 77873, 78143, 78172, 77914, 78098, 78107, 77682, 77027, 78207, 78211, 78194, 77684, 78193, 77578, 78176, 78142, 77892, 77935, 77929, 77579, 78081, 76897, 76956, 78092, 77587, 77597, 77810, 77599, 77029, 78152, 77151, 78097, 77034, 77797, 78213, 77868, 77605, 77259, 77705, 77232, 76887, 77870, 77260, 77264, 77039, 77235, 77267, 77299, 77326, 77328, 77332, 77238, 77336, 77339, 78148, 77348, 77350, 77872, 77044, 78200, 77047, 78133, 77807, 77395, 77293, 78158, 77441, 78189, 78195, 78212, 78168, 76537, 78217, 77442, 78208, 78209, 78136, 78214, 78223, 78155, 78123, 78216, 77686, 78064, 77656, 78192, 78205, 78185, 78285, 78151, 77048, 77743, 77757, 77444, 78125, 77445, 78177, 77447, 78171, 77910, 78150, 76924, 76926, 77026, 78281, 77630, 78145, 78323, 78312, 78247, 78228, 78252, 78251, 78315, 78319, 78286, 78265, 78242, 78263, 78283, 78291, 78244, 78287, 78249, 78324, 78248, 78329, 78289, 78271, 78259, 78302, 78257, 78278, 78300, 78296, 77452, 77462, 77692, 77463, 78215, 78219, 78262, 77467, 78239, 78258, 78088, 78268, 78269, 77799, 78235, 78226, 78274, 78236, 78106, 78232, 76688, 78273, 78175, 78128, 78314, 78306, 78325, 78322, 78260, 78245, 78127, 78225, 78261, 78362, 78231, 78361, 78112, 78173, 78220, 77338, 78119, 78180, 78309, 78311, 78330, 78331, 78237, 78270, 77466, 77670, 77020, 78099, 78140, 77450, 78091, 78210, 77341, 78326, 78305, 78284, 78126, 78343, 78250, 78243, 77331, 77756, 77805, 78233, 77695, 78266, 77887, 78360, 78363, 78365, 77022, 77683, 77607, 77598, 77900, 78218, 78366, 78369, 76912, 76560, 78238, 78355, 77595, 77594, 77620, 77592, 78320, 78337, 78336, 78334, 78292, 78288, 78341, 78267, 78290, 78335, 77054, 78342, 78352, 78347, 78282, 78310, 77591, 78104, 77058, 77589, 76622, 77608, 77588, 77059, 77586, 78255, 78359, 78354, 78294, 77056, 78304, 78307, 78318, 78293, 78317, 78332, 78367, 78368, 78381, 78254, 78105, 78333, 77063, 78253, 77066, 77060, 77061, 77069, 77071, 77068, 77582, 77435, 77072, 78328, 78275, 77074, 77075, 78338, 78224, 78371, 78384, 78388, 78373, 77076, 78364, 77078, 77581, 78240, 78246, 78301, 77148, 77434, 78276, 77665, 78356, 77580, 77512, 77513, 77511, 77510, 77509, 77508, 77507, 78316, 77437, 78327, 78279, 78295, 77303, 78138, 78161, 78230, 78280, 77070, 77506, 78297, 78344, 78402, 78425, 78436, 78458, 77505, 77504, 77503, 77502, 78411, 77296, 78222, 78377, 77079, 77080, 78350, 76872, 77053, 77611, 78346, 78370, 77082, 77261, 77501, 77500, 77499, 77498, 77497, 77496, 77495, 77494, 77493, 76773, 76777, 77081, 78376, 78378, 78385, 78387, 78468, 77085, 77471, 77472, 77483, 77484, 77464, 77489, 77268, 77084, 77456, 77667, 77086, 78463, 78445, 78433, 77089, 77898, 77083, 77090, 77751, 77095, 78229, 78452, 78386, 78416, 78398, 78444, 78432, 78456, 78403, 78407, 78417, 78424, 78408, 78396, 78437, 78308, 78391, 78395, 78383, 78389, 78469, 78241, 77292, 78345, 78457, 77088, 77091, 78299, 76875, 77265, 78380, 77093, 78390, 77295, 77094, 78474, 77300, 77301, 78313, 78431, 78460, 78455, 78392, 78470, 78438, 77302, 76931, 78467, 77297, 77304, 78471, 78382, 78464, 78428, 77305, 77107, 78272, 78449, 78461, 77490, 77488, 77487, 77485, 78423, 77461, 78397, 78450, 78379, 78472, 78491, 78492, 78475, 78476, 78477, 78493, 78494, 78495, 78496, 78497, 78480, 78481, 78482, 78485, 78498, 78499, 78500, 78501, 78502, 78503, 78504, 78505, 78506, 78507, 78414, 78462, 78479, 78484, 78486, 78487, 78514, 78515, 78516, 78517, 78518, 78519, 78520, 78521, 78522, 78523, 78489, 78524, 78525, 75832, 78473, 78394, 77816, 78256])
println("number of invalid cells: $(length(invalidIdSet)) ")
abandonSet = IntSet([76669, 78229, 78282, 78293, 78293, 78294, 78297, 78299, 78301, 78303, 78308, 78310, 78317, 78318])
println("number of abandon cells: $(length(abandonSet))")
for line in readlines("valid_cell_ids.txt")
    cellId, isValid = split(line)
    cellId = parse(cellId)
    if  contains(isValid,"TRUE") && !isfile("swc.bin/$(cellId).swc.bin")
        println("this neuron was not built in consensus and was not skeletonized in $(cellId)")
    end 
    if contains(isValid,"TRUE") && isfile("swc.bin/$(cellId).swc.bin") &&
        !(cellId in invalidIdSet) && !(cellId in abandonSet)
        push!(cellIdList, cellId)
    end 
end 
cellIdList

In [ ]:
@load "cell_id_list_with_soma.jld" cellIdListWithSoma
cellIdListWithSoma

In [ ]:
# evaluation
groundTruthClassSetList = Vector{Set{Int}}()
push!(groundTruthClassSetList, Set([77648,77710,77300,77705]))
push!(groundTruthClassSetList, Set([77148,77625,77641,77692]))
push!(groundTruthClassSetList, Set([76355,76487,76267,76350]))
push!(groundTruthClassSetList, Set([77805,77460,77163,77797]))
push!(groundTruthClassSetList, Set([76189,76185,76877,76829]))
# 77648    77148    76355    77805    76189
# 77710    77625    76487    77460    76185
# 77300    77641    76267    77163    76877
# 77705    77692    76350    77797    76829
@show groundTruthClassSetList
@assert 77648 in groundTruthClassSetList[1]
groundTruthCellIdList = Vector{Int}()
for k in 1:5
    push!(groundTruthCellIdList, groundTruthClassSetList[k]...)
end
groundTruthCellIdList

In [ ]:
# cellIdList = Set(cellIdList) ∪ Set(groundTruthCellIdList)
# cellIdList = [cellIdList...]
cellIdList = groundTruthCellIdList

In [ ]:
# classificationIdList = Set(cellIdListWithSoma) ∩ Set(groundTruthCellIdList) 
classificationIdSet = Set(cellIdList)

newGroundTruthClassSetList = Vector{Set{Int}}()
for (classId, cellIdSet) in enumerate(groundTruthClassSetList)
    s = cellIdSet ∩ classificationIdSet
    if !isempty(s)
        push!(newGroundTruthClassSetList, s)
    end 
end 
classificationIdList = [classificationIdSet...]
@show classificationIdSet
groundTruthClassSetList = newGroundTruthClassSetList

groundTruthClassAssignmentList = Vector{Int}()
for cellId in classificationIdSet
    for (classId, cellIdSet) in enumerate(groundTruthClassSetList)
        if cellId in cellIdSet
            push!(groundTruthClassAssignmentList, classId)
            break
        end
    end
end
@show groundTruthClassAssignmentList

In [ ]:
cellIdList = classificationIdList

In [ ]:
neuronDict = Dict{Int64, Neuron}()
for cellId in cellIdList
    if !isfile("neuron/$(cellId).jld") 
        @time neuron = Neurons.load_swc_bin("swc.bin/$(cellId).swc.bin")
        neuron = Neurons.remove_subtree_in_soma(neuron)
        neuron = Neurons.remove_hair( neuron );
        @save "neuron/$(cellId).jld" neuron
    else 
        @load "neuron/$(cellId).jld" neuron
    end
    neuronDict[cellId] = neuron
end 
length(keys(neuronDict))

In [ ]:
@parallel for cellId in cellIdList
    fileName = "density_map/$(cellId).jld"
    if !isfile( fileName )
        neuron = neuronDict[cellId]
        densityMap = Neurons.get_arbor_density_map( neuron, VOXEL_SIZE, GAUSSIAN_FILTER_STD );
    #     densityMap = Neurons.translate_soma_to_coordinate_origin(neuron, densityMap)
        # @assert norm(densityMap[:]) ≈ Neurons.get_total_path_length(neuron)
#         @assert norm(densityMap[:]) ≈ 1.0
        @save fileName densityMap
    end 
end 

In [ ]:
# cellId = 76918
# neuron = neuronDict[cellId]
# @load "density_map/$(cellId).jld" densityMap
# @show somaCorrdinate = Neurons.get_root_node(neuron)[1:3]
# @show map(x->x/400,somaCorrdinate)
# indices(densityMap)

In [ ]:
# heatmap(projection2)

In [ ]:
using PyCall
@pyimport skimage.feature.template as template
@everywhere function normalized_corr(image, template)
    bufferSize = map(is,ts -> is+ts, size(image), size(template))
    buffer = zeros(Float64, bufferSize)
    buffer[1:size(image,1), 1:size(image,2), 1:size(image,3)] = image
    corr = template.match_template(buffer, template, pad_input=true )
    return corr
end
# corr = normalized_corr(densityMap1, densityMap2)

In [ ]:
@show nworkers()
cellNum = length(cellIdList)
distanceMatrix = ones(cellNum, cellNum)

@everywhere const DENSITY_MAP_DIR = joinpath(homedir(),
    "workspace/zfish_analysis/21_morphological_analysis/1130/density_map")

@everywhere function density_map_distance_worker(jobs, results)
#         densityMap2 = Neurons.get_arbor_density_map(neuron2, VOXEL_SIZE, GAUSSIAN_FILTER_STD)
    while true 
        i,j, densityMap1, cellId2, neuron2 = take!(jobs)
        @load joinpath(DENSITY_MAP_DIR, "$(cellId2).jld") densityMap
#         densityMap2 = Neurons.translate_soma_to_coordinate_origin(neuron2, densityMap, VOXEL_SIZE)
#         d = Neurons.get_arbor_density_map_distance(densityMap1, densityMap2)
        corr = normalized_corr(densityMap1|>parent, densityMap|>parent)
        println("i: $i, j: $j, corr: $(maximum(corr))")
        d = (2.0 - maximum(corr))/4.0
        println("$i,$j => $d")
        put!(results, (i,j,d))
    end
end 


const jobs = RemoteChannel(()->Channel{Tuple}(12));
const results = RemoteChannel(()->Channel{Tuple}(12));

for i in 1:cellNum
    cellId1 = cellIdList[i]
    neuron1 = neuronDict[cellId1]
#     densityMap1 = Neurons.get_arbor_density_map(neuron1, VOXEL_SIZE, GAUSSIAN_FILTER_STD)
    @load joinpath(DENSITY_MAP_DIR, "$(cellId1).jld") densityMap
#     densityMap1 = Neurons.translate_soma_to_coordinate_origin(neuron1, densityMap, VOXEL_SIZE)
    densityMap1 = densityMap
    @sync begin 
        @async begin
            for j in i+1:cellNum
                cellId2 = cellIdList[j]
                neuron2 = neuronDict[cellId2]
                put!(jobs, (i,j, densityMap1, cellId2, neuron2))
            end
        end 
        for p in workers()
            @async remote_do(density_map_distance_worker, p, jobs, results)
        end 
        @async begin
            for j in i+1:cellNum
                x,y,d = take!(results)
                distanceMatrix[x,y] = d
                distanceMatrix[y,x] = d
            end 
        end
    end
end

# for i in 1:cellNum
#     cellId1 = cellIdList[i]
#     neuron1 = neuronDict[cellId1]
# #     densityMap1 = Neurons.get_arbor_density_map(neuron1, VOXEL_SIZE, GAUSSIAN_FILTER_STD)
#     @load "density_map/$(cellId1).jld" densityMap
#     densityMap1 = Neurons.translate_soma_to_coordinate_origin(neuron1, densityMap, VOXEL_SIZE)
#     for j in i+1:cellNum
#         cellId2 = cellIdList[j]
#         neuron2 = neuronDict[cellId2]
# #         densityMap2 = Neurons.get_arbor_density_map(neuron2, VOXEL_SIZE, GAUSSIAN_FILTER_STD)
#         @load "density_map/$(cellId2).jld" densityMap
#         densityMap2 = Neurons.translate_soma_to_coordinate_origin(neuron2, densityMap, VOXEL_SIZE)
#         d = Neurons.get_arbor_density_map_distance(densityMap1, densityMap2)
#         distanceMatrix[i,j] = d
#         distanceMatrix[j,i] = d
#         println("$i,$j : $d")
#     end 
# end

distanceMatrix

using JLD2
@save "arbor_density_map.jld" cellIdList neuronDict distanceMatrix 

In [ ]:
@show maximum(distanceMatrix)
@show minimum(distanceMatrix)

In [ ]:
@load "arbor_density_map.jld" cellIdList neuronDict distanceMatrix
# Plots.heatmap(distanceMatrix)
distanceMatrix2 = zeros(length(classificationIdList), length(classificationIdList)) 
for (i, cellId1) in enumerate(classificationIdList)
    r = findin(cellIdList, cellId1)[1]
    for (j, cellId2) in enumerate(classificationIdList)
        c = findin(cellIdList, cellId2)[1]
        distanceMatrix2[i,j] = distanceMatrix[r,c]
    end
end
distanceMatrix = distanceMatrix2

Plots.heatmap(distanceMatrix)

In [ ]:
using Plots
histogram(distanceMatrix[:])

In [ ]:
using Clustering
clust = Clustering.hclust( distanceMatrix, :average)

In [ ]:
plotly()

function indexmap(x::Vector)
    ret = Dict()
    for (i,v) in enumerate(x)
        ret[v] = i
    end 
    ret
end
function hclustplot(hc::Hclust, useheight::Bool)
    o = indexmap(hc.order)
    n = [x for x in 1:length(o)]

    pos = treepositions(hc, useheight)


    xs = []
    ys = []
    for i in 1: size(hc.merge, 1)
        x1 = pos[hc.merge[i,1]][1]
        x2 = pos[hc.merge[i,2]][1]
        append!(xs, [x1,x1,x2,x2])

        y1 = pos[hc.merge[i,1]][2]
        y2 = pos[hc.merge[i,2]][2]
        useheight ? h = hc.height[i] : h = 1
        newy = maximum([y1,y2]) + h
        append!(ys, [y1,newy,newy,y2])
    end
    return (reshape(xs, 4, size(hc.merge, 1)), reshape(ys, 4, size(hc.merge, 1)))
end

function treepositions(hc::Hclust, useheight::Bool)
    order = indexmap(hc.order)
    positions = Dict{}()
    for (k,v) in order
        positions[-k] = (v, 0)
    end
    for i in 1:size(hc.merge,1)
        xpos = mean([positions[hc.merge[i,1]][1], positions[hc.merge[i,2]][1]])
        if hc.merge[i,1] < 0 && hc.merge[i,2] < 0
            useheight ? ypos = hc.height[i] : ypos = 1
        else
            useheight ? h = hc.height[i] : h = 1
            ypos = maximum([positions[hc.merge[i,1]][2], positions[hc.merge[i,2]][2]]) + h
        end

        positions[i] = (xpos, ypos)
    end
    return positions
end

# Plots.pyplot()
Plots.plotly()
@show classificationIdList[clust.order]
Plots.plot(hclustplot(clust, true), seriestype=:path, color=:black,
    yaxis=nothing,  grid=false, legend=false,  xticks=classificationIdList[clust.order])

In [ ]:
# rearrange distance matrix according to order
reorderedDistanceMatrix = zeros(distanceMatrix)
for (i,o1) in enumerate(clust.order)
    for (j, o2) in enumerate(clust.order)
        reorderedDistanceMatrix[i,j] = distanceMatrix[o1,o2]
    end 
end 
gr()
Plots.heatmap(clust.order, clust.order, reorderedDistanceMatrix, 
               title="Arbor Density Map Distance matrix")

# Evaluation
precision, recall, and rand error

In [ ]:
numClusters = 4
classes = cutree(clust; k=numClusters)
@show classes

# clusterMap = Vector{Set{Int}}()
# for k in 1:numClusters
#     push!(clusterMap, Set{Int}())
# end
# for (i,classId) in enumerate(classes)
#     cellId = classificationIdList[i]
#     push!(clusterMap[classId], cellId)
# end 
# @show clusterMap
# @show groundTruthClassList
# @show map(length, groundTruthClassList) |> sum



adjustedRandIndex, randIndex, mirkinIndex, hubertIndex = Clustering.randindex(classes, groundTruthClassAssignmentList)
@show adjustedRandIndex, randIndex, mirkinIndex, hubertIndex
@show variationOfInformation = varinfo(4, classes, 4,groundTruthClassAssignmentList)

In [ ]:
@show clust.order
classificationIdList[clust.order]

In [ ]:
neuron = neuronDict[76292]
mask = Neurons.get_mask(neuron, (300,300,300))
PlotRecipes.plot_mask(mask |> parent)

In [ ]:
PlotRecipes.plot_arbor_density_map(densityMap1 |> parent)

In [ ]:
@show norm(densityMap[:])
@show Neurons.get_total_path_length(neuron)
# gr()
histogram(parent(densityMap)[1:40:end], nbin=100)

In [ ]:
plotly()

PlotRecipes.plot(neuron)

In [ ]:
maximum(mask.parent, 1)[1,:,:] |> Array{Gray{Bool}, 2}